<a href="https://colab.research.google.com/github/AquaFAN171029/Qwen-Fire/blob/main/Qwen_image_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip -q install diffusers>=0.35.1 transformers accelerate safetensors pillow

import os, torch
from PIL import Image, ImageDraw
from diffusers import QwenImageEditPipeline

edit = QwenImageEditPipeline.from_pretrained(
    "Qwen/Qwen-Image-Edit",
    torch_dtype=torch.bfloat16
    ).to("cuda")
print("loaded")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/512 [00:00<?, ?B/s]

Fetching 36 files:   0%|          | 0/36 [00:00<?, ?it/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

loaded


In [19]:
from PIL import Image
import numpy as np
import cv2
import os, json

base = Image.open("back.png").convert("RGB")
x1,y1,x2,y2 = 799, 643, 935, 962
pad = 12
x1p=max(0,x1-pad); y1p=max(0,y1-pad); x2p=min(base.width,x2+pad); y2p=min(base.height,y2+pad)
roi = base.crop((x1p,y1p,x2p,y2p))

#在 ROI上插入灭火器
prompt = ("Insert a photorealistic handheld fire extinguisher in the central area; "
          "red cylindrical metal body, black hose, metal nozzle, pressure gauge, "
          "generic safety label, soft contact shadow, consistent with surrounding lighting")
out_roi = edit(
    image=roi,
    prompt=prompt,
    negative_prompt="cartoon, painting, logo, watermark, blurry, deformed",
    true_cfg_scale=4.0,
    num_inference_steps=50,
    generator=torch.manual_seed(42)
).images[0]

# 转坐标到int，判定坐标范围
W, H = base.size
x1  = int(x1);  y1  = int(y1);  x2  = int(x2);  y2  = int(y2)
x1p = max(0, int(x1p)); y1p = max(0, int(y1p))
x2p = min(W, int(x2p)); y2p = min(H, int(y2p))

#以 roi 的真实尺寸，重新推导右下角
rw, rh = roi.size
x2p = x1p + rw
y2p = y1p + rh
assert x2p <= W and y2p <= H, "ROI Error"

#开始贴回：

base_np = np.array(base.convert("RGB"))

#贴回(x1p,y1p,x2p,y2p)
target_w, target_h = (x2p - x1p), (y2p - y1p)
out_roi_resized = out_roi.resize((target_w, target_h), Image.LANCZOS)
out_np = np.array(out_roi_resized.convert("RGB"))
#左上+右下同时对齐（四点约束）
assert out_np.shape[1] == target_w and out_np.shape[0] == target_h
base_np[y1p:y2p, x1p:x2p, :] = out_np


# 保存
os.makedirs("out", exist_ok=True)
composited = Image.fromarray(base_np, mode="RGB")
composited.save("out/out.png")
roi.save("out/roi.png") # 裁下来的原始ROI
out_roi.save("out/roi_ai.png") # AI生成的ROI
print("Saved")

  0%|          | 0/50 [00:00<?, ?it/s]

/tmp/ipython-input-1271565515.py:53: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  composited = Image.fromarray(base_np, mode="RGB")


Saved
